<table align="left">
  <td>
    <a target="_blank" href="https://github.com/J-TKim/T_Academy_Recommendation/blob/main/about_GD.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td> 
</table>

## GD 에 대해 간단하게 설명하기 위한 자료 입니다

In [1]:
import numpy as np

np.random.seed(42)

R: 우리가 타겟(?)으로 하는 맞춰야하는 정보!   
    
(row: User, col: Item) 이라고 생각합시다 

In [2]:
R = np.array([
    [0, 3, 2],
    [5, 1, 2],
    [4, 2, 1],
    [2, 0, 4],
])

- U: 유저의 정보를 담을 4 x 2 행렬
- I: 아이템의 정보를 담을 3 x 2 행렬 (행렬 곱을 위해 2 x 3으로 전치해서 사용)

- b: 각 행렬의 bias

In [3]:
U = np.random.normal(size=(4, 2))
I = np.random.normal(size=(3, 2))

b_U = np.zeros(4)
b_I = np.zeros(3)
b = np.mean(R[np.where(R != 0)])

Bias 확인

In [4]:
print(b_U, b_I)

[0. 0. 0. 0.] [0. 0. 0.]


In [5]:
b

2.6

모든 행렬들의 shape 확인

In [6]:
print(U, U.shape)

[[ 0.49671415 -0.1382643 ]
 [ 0.64768854  1.52302986]
 [-0.23415337 -0.23413696]
 [ 1.57921282  0.76743473]] (4, 2)


In [7]:
print(I, I.shape)

[[-0.46947439  0.54256004]
 [-0.46341769 -0.46572975]
 [ 0.24196227 -1.91328024]] (3, 2)


U와 I를 곱해주기 위해선 둘 중 한 행렬을 전치(Transpose)해 주어야 한다

In [8]:
print(I.T, I.T.shape)

[[-0.46947439 -0.46341769  0.24196227]
 [ 0.54256004 -0.46572975 -1.91328024]] (2, 3)


U행렬과, Transpose한 I행렬인 I.T를 곱하는 식

In [9]:
U.dot(I.T)

array([[-0.30821126, -0.16579233,  0.38472444],
       [ 0.52226197, -1.00947065, -2.75726675],
       [-0.01710435,  0.21755536,  0.39131333],
       [-0.32502055, -1.08925235, -1.08620779]])

곱한 식(U.dot(I.T)과 우리의 target 행렬은 같은 생김새

In [10]:
R

array([[0, 3, 2],
       [5, 1, 2],
       [4, 2, 1],
       [2, 0, 4]])

- reg_param: regularization을 위한 파라미터
- learing_rate: 학습 속도 설정을 위한 파라미터

In [11]:
reg_param, learning_rate = 0.01, 0.01

U.dot(I.T) 에서 우리가 원하는 위치의 값을 가져오는 식

In [12]:
def get_prediction(i, j):
    return b + b_U[i] + b_I[j] + U[i, :].dot(I[j, :].T)

error(오차)를 계산해서 기울기(역전파)를 계산하는 함수 => back propagation

In [13]:
def gradient(error, i, j):
    du = (error * I[j, :]) - (reg_param * U[i, :])
    di = (error * U[i, :]) - (reg_param * I[j, :])
    return du, di

위에서 구한 역전파를 기반으로 U, I행렬을 수정한다

In [14]:
def gradient_descent(i, j, rating):
    prediction = get_prediction(i, j)
    error = rating - prediction

    # updata biases
    b_U[i] += learning_rate * (error - reg_param * b_U[i])
    b_I[j] += learning_rate * (error - reg_param * b_I[j])

    # update latent feature
    du, di = gradient(error, i, j)
    U[i, :] = learning_rate * du
    I[j, :] = learning_rate * di

cost함수는 R행렬와 우리가 만든 U.dot(I.T)행렬의 차이를 계산한다

In [15]:
def cost():
    """
    rmse를 계산하는 함수
    """

    # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다
    xi, yi = R.nonzero()
    # predicted = self.get_complete_matrix()
    cost = 0
    for x, y, in zip(xi, yi):
        cost += pow(R[x, y] - get_prediction(x, y), 2)
    return np.sqrt(cost/len(xi))

- R.nonzero(): R행렬의 0이 아닌 원소가 있는 위치를 반환해주는 함수

In [16]:
R.nonzero()

(array([0, 0, 1, 1, 1, 2, 2, 2, 3, 3]), array([1, 2, 0, 1, 2, 0, 1, 2, 0, 2]))

In [17]:
xi, yi = R.nonzero()

위에서 얻은 위치를 기반으로 U, I행렬을 수정해준다

In [18]:
for i, j, in zip(xi, yi):
    
    gradient_descent(i, j, R[i, j])

In [19]:
U

array([[-1.47576716e-03,  1.16717814e-02],
       [ 5.16603559e-08, -1.23760560e-06],
       [-4.25602954e-09,  1.27870264e-09],
       [ 1.39587503e-08,  5.42199469e-09]])

In [20]:
I.T

array([[-9.87034330e-03, -1.18008251e-06, -1.97297303e-06],
       [-4.79642736e-03, -2.52004927e-06, -8.08534192e-07]])

In [21]:
print(U.dot(I.T))

[[-4.14165232e-05 -2.76719372e-08 -6.52538554e-09]
 [ 5.42617991e-09  3.05786361e-12  8.98721955e-13]
 [ 3.58752683e-11  1.80007237e-15  7.36315669e-15]
 [-1.63783861e-10 -3.01361710e-14 -3.19241060e-14]]


In [22]:
R

array([[0, 3, 2],
       [5, 1, 2],
       [4, 2, 1],
       [2, 0, 4]])

```python
for i, j, in zip(xi, yi):
    
    gradient_descent(i, j, R[i, j])
```

이 셀부터 cost()까지 계속 반복해서 눌러보면 cost()값이 대부분 줄어드는 것을 볼 수 있습니다!!

In [23]:
cost()

1.269346280792134